In [1]:
import numpy as np
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
from utils import get_train_test, get_columns

In [33]:
window_length = 44
horizon = 4
total_size = int(24*4*365*1.5)
excluded_columns = ['Datetime',
                    'Resolution code',
                    'Total Load',
                    'Most recent forecast',
                    'Most recent P10',
                    'Most recent P90',
                    'Day-ahead 6PM forecast',
                    'Day-ahead 6PM P10',
                    'Day-ahead 6PM P90',
                    'Week-ahead forecast',
                    #'Total Load Interpolated',
                    #'Year',
                    #'Month',
                    #'Day',
                    #'Hour',
                    #'Minute',
                    #'Season',
                    #'Lockdown',
                    'Difference with previous load',
                    #'Difference with previous load interpolated',
                    'Total Load Interpolated Persistence',
                    'Total Load Persistence'
                   ]

In [34]:
X_train, Y_train, X_test, Y_test = get_train_test(window_length, horizon, total_size, excluded_columns)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [35]:
print("Training Input shape:", X_train.shape)
print("Testing Input shape:", X_test.shape)
print("Training Output shape:", Y_train.shape)
print("Training Output shape:", Y_test.shape)

Training Input shape: (12382, 396)
Testing Input shape: (744, 396)
Training Output shape: (12382, 4)
Training Output shape: (744, 4)


In [36]:
from xgboost import XGBRegressor
xgb = XGBRegressor(tree_method="hist", multi_strategy="multi_output_tree").fit(X_train, Y_train)
y_hat_xgb = xgb.predict(X_test)[:, :4]
pd.DataFrame({"Prediction": y_hat_xgb.flatten()}).to_csv("xgb.csv")

In [37]:
!python3 evaluate.py xgb.csv

|Model Name|RMSE t+1|RMSE t+2|RMSE t+3|RMSE t+4|MAPE t+1|MAPE t+2|MAPE t+3|MAPE t+4|
| |91.8003|125.230|149.717|173.781|0.00880|0.01163|0.01376|0.01643|


In [45]:
from sklearn.ensemble import ExtraTreesRegressor
et = ExtraTreesRegressor(n_estimators=500).fit(X_train, Y_train)
y_hat_et = et.predict(X_test)[:, :4]
pd.DataFrame({"Prediction": y_hat_et.flatten()}).to_csv("et.csv")

In [46]:
!python3 evaluate.py et.csv

|Model Name|RMSE t+1|RMSE t+2|RMSE t+3|RMSE t+4|MAPE t+1|MAPE t+2|MAPE t+3|MAPE t+4|
| |84.4739|116.210|139.626|161.983|0.00810|0.01089|0.01302|0.01510|


In [40]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor().fit(X_train, Y_train)
y_hat_rf = rf.predict(X_test)[:, :4]

In [41]:
pd.DataFrame({"Prediction": y_hat_rf.flatten()}).to_csv("rf.csv")

In [42]:
!python3 evaluate.py rf.csv

|Model Name|RMSE t+1|RMSE t+2|RMSE t+3|RMSE t+4|MAPE t+1|MAPE t+2|MAPE t+3|MAPE t+4|
| |86.9795|120.300|144.058|166.529|0.00831|0.01121|0.01337|0.01556|
